## Interate agreement

In [55]:
import pandas as pd

In [56]:
excels = ['Drug-disease relations annotations - Medical Expert 1.xlsx',
          'Drug-disease relations annotations - Medical Expert 2.xlsx',
          'Drug-disease relations annotations - Medical Expert 3.xlsx']
sheets = ['Indication_treatment','indication_symptomatic_relief','contraindication', 'effect']

In [57]:
tables = []
for excel in excels:
    for sheet in sheets:
        df = pd.read_excel(excel, sheet_name=sheet)
        df['expert'] = excel[37:53]
        df['sheet'] = sheet
        tables.append(df)

In [58]:
#for i in range(12):
#    print(tables[i].columns)

In [121]:
data = pd.DataFrame()
for table in tables:
    data = data.append(table, ignore_index=True, sort=False)
#data.dropna(inplace=True)

In [122]:
#data = data.head(5).append(data.tail(5))

In [123]:
#data

In [124]:
data1 = data.head(5)['workers_answers'].str.split('\n',expand=True).stack()

In [126]:
#data1

In [ ]:
data.head(5).merge(data1.rename('new'),left_index=True, right_index=True, how='outer')

In [95]:
table = pd.DataFrame()
for row in data.iterrows(): 
    if '\n' in row[1].workers_answers:
        str(row[1].workers_answers).split('\n')
        print(row[1].workers_answers, str(row[1].workers_answers).split('\n'))
    else: 
        print(row[0], row[1])

indication_treatment
indication_treatment
indication_treatment ['indication_treatment', 'indication_treatment', 'indication_treatment']
indication_treatment
indication_treatment
indication_treatment ['indication_treatment', 'indication_treatment', 'indication_treatment']
indication_treatment
indication_treatment
indication_treatment ['indication_treatment', 'indication_treatment', 'indication_treatment']
indication_treatment
indication_treatment
indication_treatment ['indication_treatment', 'indication_treatment', 'indication_treatment']
indication_treatment
indication_treatment
indication_treatment ['indication_treatment', 'indication_treatment', 'indication_treatment']
2236 #                               NaN
context                         NaN
disease_name                    NaN
drug_name                       NaN
workers_answers              Effect
medical_expert                  NaN
do_id                           NaN
drug_brand_name                 NaN
drug_id                    

---
## Experts tables

In [28]:
data.expert.unique()

array(['Medical Expert 1', 'Medical Expert 2', 'Medical Expert 3'],
      dtype=object)

In [29]:
data = data.merge(pd.read_csv('Expert2Relation_Mapping.csv'), 
           how='left', left_on ='medical_expert', right_on ='Text') #add the correct labels

In [30]:
data.Relation.unique()

array(['Indication: Treatment', 'Indication: Symptomatic Relief',
       'Contraindication', 'Not Found', nan, 'Effect', 'Inconsistent',
       'IDK'], dtype=object)

In [36]:
experts_answers = data[['disease_name','drug_name','context','expert','Relation']]\
    .rename(columns={'Relation':'answer'})\
    .dropna()

In [37]:
experts_answers['value'] = 1

In [38]:
experts_answers.drop_duplicates(keep='first', inplace=True)

In [42]:
inter_rate_table = pd.pivot_table(
    experts_answers,
    values='value',
    index=['disease_name','drug_name','answer','context'],
    columns=['expert'],
    aggfunc='sum'
).reset_index()

In [44]:
inter_rate_table['inter_rate'] = inter_rate_table.sum(axis=1)/3

In [46]:
inter_rate_table.head()

expert,disease_name,drug_name,answer,context,Medical Expert 1,Medical Expert 2,Medical Expert 3,inter_rate
0,ACNE,TAZAROTENE,Indication: Treatment,Retinoids may cause fetal harm when administer...,1.0,NaN,1.0,0.666667
1,ACUTE CERVICITIS,LEVONORGESTREL,Contraindication,['The use of LILETTA is contraindicated when o...,1.0,NaN,1.0,0.666667
2,ACUTE INTERMITTENT PORPHYRIA,Hemin,Indication: Symptomatic Relief,PANHEMATIN is a hemin for injection indicated ...,1.0,NaN,1.0,0.666667
3,ACUTE INTERMITTENT PORPHYRIA,Hemin,Indication: Treatment,PANHEMATIN is a hemin for injection indicated ...,NaN,1.0,NaN,0.333333
4,ACUTE LYMPHOBLASTIC LEUKEMIA,Vincristine,Indication: Treatment,Marqibo is a vinca alkaloid indicated for the ...,1.0,1.0,1.0,1.000000


In [53]:
inter_rate_table.shape

(537, 8)

In [54]:
inter_rate_table[(inter_rate_table['disease_name']=='AIDS') & (inter_rate_table['drug_name']=='ALITRETINOIN')]

expert,disease_name,drug_name,answer,context,Medical Expert 1,Medical Expert 2,Medical Expert 3,inter_rate
16,AIDS,ALITRETINOIN,IDK,Panretin gel is indicated for topical treatmen...,NaN,NaN,1.0,0.333333
17,AIDS,ALITRETINOIN,Inconsistent,Panretin gel is indicated for topical treatmen...,NaN,1.0,NaN,0.333333
18,AIDS,ALITRETINOIN,Indication: Symptomatic Relief,Panretin gel is indicated for topical treatmen...,1.0,NaN,NaN,0.333333


In [51]:
inter_rate_table.groupby(['disease_name','drug_name']).count()['answer'].reset_index()

,disease_name,drug_name,answer
0,ACNE,TAZAROTENE,1
1,ACUTE CERVICITIS,LEVONORGESTREL,1
2,ACUTE INTERMITTENT PORPHYRIA,Hemin,2
3,ACUTE LYMPHOBLASTIC LEUKEMIA,Vincristine,1
4,ACUTE LYMPHOCYTIC LEUKEMIA,CYTARABINE,1
5,ACUTE MYELOBLASTIC LEUKEMIA,DOXORUBICIN,1
6,ACUTE MYELOID LEUKEMIA,IDARUBICIN,1
7,ACUTE PANCREATITIS,Cleviprex,3
8,ACUTE RESPIRATORY FAILURE,SOMATROPIN,1
9,ACUTE RHINITIS,Aceclofenac,2


In [47]:
inter_rate_table.groupby('answer').sum()

expert,Medical Expert 1,Medical Expert 2,Medical Expert 3,inter_rate
answer,,,,
Contraindication,105.0,29.0,82.0,72.000000
Effect,69.0,1.0,82.0,50.666667
IDK,0.0,1.0,15.0,5.333333
Inconsistent,0.0,26.0,13.0,13.000000
Indication: Symptomatic Relief,81.0,24.0,59.0,54.666667
Indication: Treatment,128.0,119.0,82.0,109.666667
Not Found,1.0,0.0,1.0,0.666667


In [14]:
summary = inter_rate_table.groupby('inter_rate').count().reset_index()[['inter_rate','answer']]
summary['rate'] = summary['answer']/summary['answer'].sum()*100

In [15]:
summary

expert,inter_rate,answer,rate
0,0.333333,205,43.617021
1,0.666667,168,35.744681
2,1.000000,97,20.638298


In [16]:
inter_rate_table.shape

(470, 6)

In [17]:
inter_rate_table.groupby('inter_rate').count().drop(columns=['answer','context'])

expert,Medical Expert 1,Medical Expert 2,Medical Expert 3
inter_rate,,,
0.333333,90,35,80
0.666667,157,54,125
1.000000,97,97,97
